# Simple Data Encryption Standards


The process of encrypting a plan text into an encrypted message with the use of S-DES has been
divided into multi-steps which may help you to understand it as easily as possible.
Points should be remembered.
1. It is a block cipher.
2. It has 8-bits block size of plain text or cipher text.
3. It uses 10-bits key size for encryption.
4. It is a symmetric cipher.
5. It has Two Rounds.

In [1]:
def permute(original, permutation):
    return ''.join(original[i - 1] for i in permutation)

def generate_round_keys(key):
    p10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6] #permutation table
    p8 = [6, 3, 7, 4, 8, 5, 10, 9]
   
    key = permute(key, p10)#key = 10 bit
   
    left_half = key[:5]
    right_half = key[5:]
   
    # Left circular shift
    left_half = left_half[1:] + left_half[0]
    right_half = right_half[1:] + right_half[0]
   
    round_key1 = permute(left_half + right_half, p8)
   
    # Left circular shift again
    left_half = left_half[2:] + left_half[:2]
    right_half = right_half[2:] + right_half[:2]
   
    round_key2 = permute(left_half + right_half, p8)
   
    return round_key1, round_key2

def initial_permutation(plaintext):
    ip = [2, 6, 3, 1, 4, 8, 5, 7]
    return permute(plaintext, ip)

def inverse_initial_permutation(ciphertext):
    ip_inv = [4, 1, 3, 5, 7, 2, 8, 6]
    return permute(ciphertext, ip_inv)

def f_function(right_half, round_key):
    expansion = [4, 1, 2, 3, 2, 3, 4, 1]
    p4 = [2, 4, 3, 1]
   
    expanded = permute(right_half, expansion)
    xor_result = ''.join(str(int(a) ^ int(b)) for a, b in zip(expanded, round_key))
   
    sboxes = [
        [['00', '01', '10', '11'], ['10', '00', '01', '11'], ['11', '00', '01', '00'], ['10', '01', '00', '11']],
        [['00', '01', '10', '11'], ['10', '00', '01', '11'], ['11', '00', '01', '00'], ['10', '01', '00', '11']]
    ]
   
    sbox_output = ''
    for i in range(2):
        row = int(xor_result[i * 4] + xor_result[i * 4 + 3], 2)
        col = int(xor_result[i * 4 + 1:i * 4 + 3], 2)
        sbox_output += sboxes[i][row][col]
   
    return permute(sbox_output, p4)

def round_function(left_half, right_half, round_key):
    new_right = ''.join(str(int(a) ^ int(b)) for a, b in zip(right_half, f_function(right_half, round_key)))
    new_left = right_half
    return new_left, new_right

def sdes_encrypt(plaintext, key):
    round_key1, round_key2 = generate_round_keys(key)
    plaintext = initial_permutation(plaintext)
   
    left_half = plaintext[:4]
    right_half = plaintext[4:]
   
    left_half, right_half = round_function(left_half, right_half, round_key1)
    left_half, right_half = right_half, left_half
    left_half, right_half = round_function(left_half, right_half, round_key2)
   
    ciphertext = left_half + right_half
    ciphertext = inverse_initial_permutation(ciphertext)
   
    return ciphertext

def sdes_decrypt(ciphertext, key):
    round_key1, round_key2 = generate_round_keys(key)
    ciphertext = initial_permutation(ciphertext)
   
    left_half = ciphertext[:4]
    right_half = ciphertext[4:]
   
    left_half, right_half = round_function(left_half, right_half, round_key2)
    left_half, right_half = right_half, left_half
    left_half, right_half = round_function(left_half, right_half, round_key1)
   
    plaintext = left_half + right_half
    plaintext = inverse_initial_permutation(plaintext)
   
    return plaintext

def text_to_binary(text):
    binary_string = ""
    for char in text:
        binary_char = bin(ord(char))[2:].zfill(8)
        binary_string += binary_char
    return binary_string

# Take user input for plaintext and key as words
plaintext_word = input("Enter the plaintext word: ")
key_word = input("Enter the key word: ")

# Convert words to binary
plaintext = text_to_binary(plaintext_word)
key = text_to_binary(key_word)

encrypted = sdes_encrypt(plaintext, key)
decrypted = sdes_decrypt(encrypted, key)

print("Plaintext (binary):", plaintext)
print("Key (binary):", key)
print("Encrypted:", encrypted)
print("Decrypted:", decrypted)

Enter the plaintext word:  rubanginosingh
Enter the key word:  ruban


Plaintext (binary): 0111001001110101011000100110000101101110011001110110100101101110011011110111001101101001011011100110011101101000
Key (binary): 0111001001110101011000100110000101101110
Encrypted: 11001001
Decrypted: 00111111
